Para esta función se necesita:

- Desde el Dataframe steam_games el id de juego y la fecha de lanzamiento.
- Desde el Dataframe users_reviews el id de juego y la columna de sentimiento.

Con estos datos se crea el Dataframe que se sube para la consulta

In [1]:
# Importar las librerías a utilizar
import pandas as pd

In [2]:
# Cargar los Dataframes
df_user_reviews = pd.read_csv("C:/Users/Usuario/Desktop/Varios yo/SoyHenry/P1/Henry_PI1_Steam/data/user_reviews_2.csv")
df_steam_games = pd.read_csv("C:/Users/Usuario/Desktop/Varios yo/SoyHenry/P1/Henry_PI1_Steam/data/steam_games_2.csv")

## TRANSFORMACIONES de steam_games

In [3]:
# Revisar las columnas de steam_games
df_steam_games.columns

Index(['Unnamed: 0', 'app_name', 'release_date', 'price', 'early_access', 'id',
       'developer', 'genre', 'tag', 'spec'],
      dtype='object')

In [4]:
# Desde el Dataframe df_steam_games eliminar las columnas que no se necesitan
df_steam_games.drop("Unnamed: 0", axis=1, inplace=True)
df_steam_games.drop("app_name", axis=1, inplace=True)
df_steam_games.drop("price", axis=1, inplace=True)
df_steam_games.drop("early_access", axis=1, inplace=True)
df_steam_games.drop("developer", axis=1, inplace=True)
df_steam_games.drop("genre", axis=1, inplace=True)
df_steam_games.drop("tag", axis=1, inplace=True)
df_steam_games.drop("spec", axis=1, inplace=True)
df_steam_games

,release_date,id
0,2018-01-04,761140.0
1,2018-01-04,761140.0
2,2018-01-04,761140.0
3,2018-01-04,761140.0
4,2018-01-04,761140.0
...,...,...
2088821,NaN,681550.0
2088822,NaN,681550.0
2088823,NaN,681550.0
2088824,NaN,681550.0


In [5]:
# Debido a la presenca de "tag" y "spec" ahora eliminadas, quedaron muchos duplicados
df_steam_games.drop_duplicates(inplace=True)

In [6]:
# Revisar el Dataframe
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32134 entries, 0 to 2088778
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  29782 non-null  object 
 1   id            32132 non-null  float64
dtypes: float64(1), object(1)
memory usage: 753.1+ KB


In [7]:
# Convertir la columna 'release_date' a objetos de fecha con el formato 'YYYY-MM-DD'
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], format='%Y-%m-%d', errors='coerce')

In [8]:
# Extraer el año desde la fecha creada
df_steam_games['release_date'] = df_steam_games['release_date'].dt.year

In [10]:
df_steam_games['release_date'].unique()

array([2018., 2017.,   nan, 1997., 1998., 2016., 2006., 2005., 2003.,
       2007., 2002., 2000., 1995., 1996., 1994., 2001., 1993., 2004.,
       1999., 2008., 2009., 1992., 1989., 2010., 2011., 2013., 2012.,
       2014., 1983., 1984., 2015., 1990., 1988., 1991., 1985., 1982.,
       1987., 1981., 1986., 2021., 2019., 1975., 1970., 1980.])

En este caso, como en consultas anteriores, debido a que el consumidor de la api le pasa un año, no tiene sentido guardar registros de años NaN, porque nunca serán consumidos.

In [11]:
df_steam_games.dropna(subset="release_date", inplace=True)

In [13]:
# Convertir el año en un número entero
df_steam_games['release_date'] = df_steam_games['release_date'].astype(int)

In [14]:
df_steam_games

,release_date,id
0,2018,761140.0
25,2018,643980.0
557,2017,670290.0
707,2017,767400.0
740,2018,772540.0
...,...,...
2088578,2018,745400.0
2088638,2018,773640.0
2088670,2018,733530.0
2088715,2018,610660.0


## TRANSFORMACIONES de user_reviews

In [15]:
# Revisar las columnas de user_reviews
df_user_reviews.columns

Index(['user_id', 'posted', 'item_id', 'recommend', 'polarity', 'sentiment'], dtype='object')

In [16]:
# Desde el Dataframe df_user_reviews eliminar las columnas que no se necesitan
df_user_reviews.drop("user_id", axis=1, inplace=True)
df_user_reviews.drop("posted", axis=1, inplace=True)
df_user_reviews.drop("recommend", axis=1, inplace=True)
df_user_reviews.drop("polarity", axis=1, inplace=True)
df_user_reviews

,item_id,sentiment
0,1250.0,2
1,22200.0,2
2,43110.0,2
3,251610.0,2
4,227300.0,0
...,...,...
59328,70.0,2
59329,362890.0,2
59330,273110.0,2
59331,730.0,2


In [17]:
# Renombrar la columna con el id de juego
df_user_reviews.rename(columns={'item_id': 'id'}, inplace=True)

--------------------------

## Ahora con los Dataframes procesados, creo el csv para la consulta

In [18]:
# Agregar al Dataframe de sentimiento, la fecha de lanzamiento
df_sentimiento = df_user_reviews.merge(df_steam_games, on='id', how='left')

In [19]:
df_sentimiento

,id,sentiment,release_date
0,1250.0,2,2009.0
1,22200.0,2,2009.0
2,43110.0,2,NaN
3,251610.0,2,NaN
4,227300.0,0,2013.0
...,...,...,...
59328,70.0,2,1998.0
59329,362890.0,2,2015.0
59330,273110.0,2,2014.0
59331,730.0,2,2012.0


In [20]:
# Borrar el id, que ya no es necesario
df_sentimiento.drop("id", axis=1, inplace=True)
df_sentimiento

,sentiment,release_date
0,2,2009.0
1,2,2009.0
2,2,NaN
3,2,NaN
4,0,2013.0
...,...,...
59328,2,1998.0
59329,2,2015.0
59330,2,2014.0
59331,2,2012.0


In [21]:
# Borrar los NaN de fecha de lanzamiento
df_sentimiento.dropna(subset="release_date", inplace=True)
df_sentimiento

,sentiment,release_date
0,2,2009.0
1,2,2009.0
4,0,2013.0
5,0,2013.0
6,2,2013.0
...,...,...
59328,2,1998.0
59329,2,2015.0
59330,2,2014.0
59331,2,2012.0


In [23]:
# Crear una tabla cruzada (crosstab) para contar los valores de sentimiento por año
crosstab_recuento = pd.crosstab(df_sentimiento['release_date'], df_sentimiento['sentiment'])

In [24]:
crosstab_recuento

sentiment,0,1,2
release_date,,,
1989.0,0,0,1
1990.0,1,0,4
1991.0,0,0,1
1992.0,0,0,3
1993.0,3,0,2
1994.0,4,2,9
1995.0,4,2,10
1996.0,3,0,5
1997.0,7,11,30


In [25]:
# Renombrar las columnas
crosstab_recuento.columns = ['Negativo', 'Neutro', 'Positivo']

crosstab_recuento

,Negativo,Neutro,Positivo
release_date,,,
1989.0,0,0,1
1990.0,1,0,4
1991.0,0,0,1
1992.0,0,0,3
1993.0,3,0,2
1994.0,4,2,9
1995.0,4,2,10
1996.0,3,0,5
1997.0,7,11,30


In [31]:
crosstab_recuento.columns

Index(['Negativo', 'Neutro', 'Positivo'], dtype='object')

In [44]:
crosstab_recuento.index

Index([1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0,
       1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0,
       2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0,
       2016.0, 2017.0],
      dtype='float64', name='release_date')

In [45]:
# Exportar el Dataframe a .csv
crosstab_recuento.to_csv("sentiment_analysis.csv")

In [46]:
# Recargar el .csv
df_recuento = pd.read_csv("sentiment_analysis.csv")
df_recuento

,release_date,Negativo,Neutro,Positivo
0,1989.0,0,0,1
1,1990.0,1,0,4
2,1991.0,0,0,1
3,1992.0,0,0,3
4,1993.0,3,0,2
5,1994.0,4,2,9
6,1995.0,4,2,10
7,1996.0,3,0,5
8,1997.0,7,11,30
9,1998.0,21,18,57


In [47]:
# Convertir a tipo de dato entero la columna del año de lanzamiento
df_recuento['release_date'] = df_recuento['release_date'].astype(int)
df_recuento

,release_date,Negativo,Neutro,Positivo
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,0,0,3
4,1993,3,0,2
5,1994,4,2,9
6,1995,4,2,10
7,1996,3,0,5
8,1997,7,11,30
9,1998,21,18,57


In [49]:
# Exportar el Dataframe a .csv eliminando el índice
df_recuento.to_csv("sentiment_analysis.csv", index=False)

In [50]:
# Verificar el .csv
df = pd.read_csv("sentiment_analysis.csv")
df

,release_date,Negativo,Neutro,Positivo
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,0,0,3
4,1993,3,0,2
5,1994,4,2,9
6,1995,4,2,10
7,1996,3,0,5
8,1997,7,11,30
9,1998,21,18,57
